# NREL 5MW Temporal Conv Net (TCN) Surrogate model

This notebook is the initial try to build a TCN surrogate model on NREL 5MW aero-elastic simulations.


*   The wind is uniform over the rotor, the input to the model is HH wind time seris (U)
*   Normalization using MaxMin scaling

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)



# Mounting Google Drive
All the simulations are save in CSV files and saved on the google drive.

## Installing and Loading packages
Some packages are required for this work. Mainly, the TCN package developed by  Philippe Rémy (https://github.com/philipperemy/keras-tcn).

In [5]:
from google.colab import drive
import os
drive.mount('/content/drive/')



Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Change the current working directory
To make life easier, the current directory changed to the folder.

In [6]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/NREL5MW_TCN_DD_UniWind')
print(os.getcwd())

/content/drive/MyDrive/Colab Notebooks/NREL5MW_TCN_DD_UniWind


KERAS-TCN installation

In [ ]:
!pip3 install keras-tcn

I like to have a latex compiler for the figures. Here is the package.

In [ ]:
!sudo apt install cm-super dvipng texlive-latex-extra texlive-latex-recommended


Other required packges are called in this cell. Also the fonts are set for the figures.

In [ ]:
import tensorflow as tf
import os
import numpy
import time
from utilities import select_gpus, plot_results # utilities.py: Contains a few miscellaneous functions 
#from tcnae import TCNAE # tcnae.py: Specification of the TCN-AE model
#import data # data.py: Allows to generate anomalous Mackey-Glass (MG) time series
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import h5py


import time as tm
import tensorflow
from tcn import TCN, tcn_full_summary


from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import UpSampling1D
from tensorflow.keras.layers import AveragePooling1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential

import random



plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('font', size = 30)
plt.rc('xtick', labelsize = 25)
plt.rc('ytick', labelsize = 25)
plt.rc('legend',fontsize=25) # using a size in points


## Test GPU existance and employe one

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))



# Data import from CSV saved before

Read the data channels and save them in a `dictionary` format.

In [ ]:
# Reading the wind speeds
raw_data = {}
raw_data['wspdX'] = pd.read_csv('data_fullDLC12/WindSpeedX.csv').transpose()
raw_data['MyB'] = pd.read_csv('data_fullDLC12/RootMyb.csv').transpose()
raw_data['MxB'] = pd.read_csv('data_fullDLC12/RootMxb.csv').transpose()
raw_data['MxTB'] = pd.read_csv('data_fullDLC12/TwrBsMy.csv').transpose()
raw_data['MyTB'] = pd.read_csv('data_fullDLC12/TwrBsMy.csv').transpose()
raw_data['FxTT'] = pd.read_csv('data_fullDLC12/YawBrFxp.csv').transpose()
raw_data['FyTT'] = pd.read_csv('data_fullDLC12/YawBrFyp.csv').transpose()
raw_data['MxTT'] = pd.read_csv('data_fullDLC12/YawBrMxp.csv').transpose()
raw_data['MyTT'] = pd.read_csv('data_fullDLC12/YawBrMyp.csv').transpose()






# Nested dictionaries
Building the nested dictionries from the importated data, and normalizing the input-output data based on the MaxMin scaling

In [ ]:
chls = ['wspdX','MxB','MyB','FxTT','FyTT','MxTT','MyTT','MxTB','MyTB']
mps = ['03mps','04mps','05mps','06mps','07mps','08mps','09mps','10mps','11mps','12mps','13mps','14mps','15mps','16mps','17mps','18mps','19mps','20mps','21mps','22mps','23mps','24mps','25mps']

data_sets={}
data_sets_norm={}

for ch in chls:
  inx_lst = raw_data[ch].index.to_list()
  data_sets[ch] = {}
  data_sets_norm[ch] ={}
  for u in mps:
    indices = [i for i, s in enumerate(inx_lst) if u in s]
    data_sets[ch][u] = raw_data[ch].iloc[indices]
    data_sets_norm[ch][u] =data_sets[ch][u].subtract(data_sets[ch][u].min().min(),axis=0).div((data_sets[ch][u].max().max()-data_sets[ch][u].min().min()),axis=0)
    del indices





#### Output normalization visual check

In [ ]:
u = '22mps'
idx = 12 #This is an arbitary number to present one of the seeds as an example. It can be anything between 0-90.

fig,ax = plt.subplots(4,1,sharex=True)
ax[0].plot(data_sets_norm['wspdX'][u].iloc[idx])
ax[1].plot(data_sets_norm['MxB'][u].iloc[idx])
ax[2].plot(data_sets_norm['MyB'][u].iloc[idx])
ax[3].plot(data_sets_norm['MxTB'][u].iloc[idx])


# TCN-AE trial for the wind input and loads output

What I am going to try here is training the TCN-AE network on the different input output instead of the same input-output. I want to see if things are going to work. 

In [ ]:
import tcnae
import imp
imp.reload(tcnae)

# This is for the naming convention of the saved figures
timestr = tm.strftime('%Y%m%d')

# Time for x axis
time = np.linspace(0,600,12000)

# For every wind speed we have 90 seeds
seeds = np.arange(0,90)

# The x-axis for the loss plot
epoch_xaxis=np.arange(1,301)

# Num of seeds used for the training
TrnSzSd = 40

# The wind speeds the training is going to be done
U=['24mps']

# The channels/sensors the training is going to be done
CH=['MyTB']

# Dict to save the history of training
hist_data_set = {}

#Dict to save the the predictions values
ypred={}

for u in U:
  hist_data_set[u] = {}
  ypred[u]={}
  for ch in CH:
    print(u,ch)
    # Choose seeds out of 90 seeds for training
    rnd_seed = np.random.random_integers(1, high=90, size=TrnSzSd)
    # Find the remaninag seeds and use them later for prediction and choose 
    # one random
    rem_seeds = seeds[~np.in1d(np.arange(seeds.size),rnd_seed)]
    SeedNo=random.choice(rem_seeds)
    # Dedicate input and output to x and y for training
    x = data_sets_norm['wspdX'][u].iloc[rnd_seed,1:]
    y = data_sets_norm[ch][u].iloc[rnd_seed,1:]
    # Build the model based on the setting in the 'tcnae.py' file
    tcn_ae = tcnae.TCNAE()
    tcn_ae.build_model()
    # Fit the model to the traininag data
    history = tcn_ae.model.fit(x, y, 
                            batch_size=12, 
                            validation_split=0.1, 
                            shuffle=True,
                            verbose=1,
                            epochs = 300)
    # Keep thr history
    hist_data_set[u][ch] = history
    # Prediction with the random seed
    y_pred = tcn_ae.model.predict(data_sets_norm['wspdX'][u].iloc[SeedNo:SeedNo+1,1:])
    # Saving the prediction
    ypred[u][ch] = y_pred
    # Plotting
    fig = plt.figure(figsize=(20,14))
    gs = fig.add_gridspec(2,2)
    ax1 = fig.add_subplot(gs[0, :])
    ax2 = fig.add_subplot(gs[1, 0])
    ax3 = fig.add_subplot(gs[1, 1])
    ax1.semilogy(epoch_xaxis,history.history['loss'],linewidth=2,label='Training Loss')
    ax1.semilogy(epoch_xaxis,history.history['val_loss'],linewidth=2,label='Validation Loss')
    ax1.set_xlabel('Epoch [-]')
    ax1.set_ylabel('MSE [-]')
    ax1.grid()
    ax1.legend()
    ax2.plot(time, data_sets_norm[ch][u].iloc[SeedNo,1:],label ='Simulation',linewidth=2)
    ax2.plot(time,y_pred[0,:,:],label='Prediction',alpha=0.7,linewidth=2)
    ax2.set_ylabel('Scaled '+ch+' [-]')
    ax2.grid()
    ax2.legend()
    ax2.set_xlabel('Time [s]')
    a = ax3.hist(data_sets_norm[ch][u].iloc[SeedNo,1:], density=True, bins =20, orientation='horizontal',label ='Simulation',alpha = 0.75)
    _ = ax3.hist(y_pred[0,:,0],label='Prediction',bins = a[1], orientation='horizontal',alpha = 0.5,density=True)
    ax3.grid()
    ax3.legend()
    ax3.set_xlabel('Probablity [-]')
    fig.savefig(timestr+'_TCN_EncDec_UniWind_'+u+'_'+'Norm'+ch+'.png', dpi=400, facecolor='w',
             edgecolor='w', orientation='portrait', format='png', transparent=False)
  







# Testing: The hyper-parameter tuning/testing rig

This piece of code is used to generate plots, while the hyper parameters in the `tcnae.py` code are changed. This was necessary to find an acceptable level of accuracy and keeping the track of what I was doing. The trainging part is not here, as it is a repeat of the loop above.



In [ ]:
timestr = tm.strftime('%Y%m%d%H%M%S')
time = np.linspace(0,600,12000)
seeds = np.arange(0,90)
rem_seeds = seeds[~np.in1d(np.arange(seeds.size),rnd_seed)]
SeedNo=random.choice(rem_seeds)
#rem_seeds = a[~np.in1d(np.arange(a.size),r)]

#ch = 'MyTB'
#u = '10mps'


d = { 'a': str(tcn_ae.dilations),
      'b': str(tcn_ae.nb_filters),
      'c': str(tcn_ae.kernel_size),
      'd': str(tcn_ae.filters_conv1d),
      'e': str(tcn_ae.activation_conv1d),
      'f': str(tcn_ae.latent_sample_rate),
      'g': str(tcn_ae.tcn_activation),
      'h': str(tcn_ae.nb_stacks)}

s = "dilations={a}\n nb_filters={b}\n kernel_size={c}\n filters_conv1d={d}\n activation_conv1d={e}\n latent_sample_rate={f}\n tcn_activation={g}\n nbs={h}"
s.format(**d)


y_pred = tcn_ae.model.predict(data_sets_norm['wspdX'][u].iloc[SeedNo:SeedNo+1,1:])
#y_pred = loaded_model.predict(data_sets_norm['wspdX'][u].iloc[SeedNo:SeedNo+1,1:])

print(y_pred.shape)

fig,ax = plt.subplots(2,2,figsize=(20,14))

ax=ax.flatten()

ax[0].plot(time,data_sets_norm['wspdX'][u].iloc[SeedNo,1:])
ax[0].grid()
ax[0].set_ylabel('Normalized uniform wind speed [-]')

ax[1].hist(data_sets_norm['wspdX'][u].iloc[SeedNo,1:],bins=30,density=True, orientation=u'horizontal')
ax[1].grid()


ax[2].plot(time, data_sets_norm[ch][u].iloc[SeedNo,1:],label ='Normalized Simulation')
ax[2].plot(time,y_pred[0,:,:],label='Normalized Prediction',alpha=0.6)
ax[2].set_ylabel('Normalized My @ Tower Bottom [-]')
ax[2].grid()
ax[2].legend()
ax[2].set_xlabel('Time [s]')

a = ax[3].hist(data_sets_norm[ch][u].iloc[SeedNo,1:], density=True, bins =20, orientation='horizontal',label ='Normalized Simulation',alpha = 0.75)
_ = ax[3].hist(y_pred[0,:,0],label='Prediction',bins = a[1], orientation='horizontal',alpha = 0.5,density=True)
ax[3].grid()
ax[3].legend()
ax[3].set_xlabel('Probablity [-]')
plt.text(0.5,0.5,s.format(**d))
fig.savefig(timestr+'_TCN_EncDec_UniWind_'+u+'_'+'Norm'+ch+'.png', dpi=400, facecolor='w',
             edgecolor='w', orientation='portrait', format='png', transparent=False)